# Demo Notebook Exciting Slurm Calculation
\
Show basic use cases of the exciting calculation class defined inside the ExcitingWorkflow package.\
Source code: [Gitlab ExcitingWorkflow](https://git.physik.hu-berlin.de/peschelf/excitingworkflow)\
\
Run this notebook on dune!

In [1]:
import os
import time
import shutil

from excitingworkflow.src.exciting_slurm_calculation import ExcitingSlurmCalculation
from excitingtools.input.ground_state import ExcitingGroundStateInput
from excitingtools.input.structure import ExcitingStructure
from excitingtools.input.xs import ExcitingXSInput
from excitingtools.runner import BinaryRunner
from exgw.src.job_schedulers import slurm

In [2]:
os.system('rm -r calculation*')  # pay attention here!!
print(os.listdir()) # no calculation directories here

['.ipynb_checkpoints', 'old_Demo_Exciting_Slurm_Calculation.ipynb', 'Demo_Exciting_Slurm_Calculation.ipynb']


In [3]:
os.system('sinfo')

           PARTITION AVAIL  TIMELIMIT   NODES(A/I/O/T) NODELIST
      cpu16memory128    up 30-00:00:0        0/38/6/44 node[133-176]
      cpu32memory512    up 30-00:00:0          0/4/0/4 node[177-180]
      cpu36memory192    up 30-00:00:0        4/12/0/16 node[181-196]
      cpu36memory384    up 30-00:00:0         5/7/0/12 node[197-208]
               debug    up    1:00:00          0/2/0/2 node[109-110]
               test*    up 1-00:00:00          0/2/0/2 node[111-112]


0

### Exciting Groundstate calculation of LiF
Define the necessary input objects for an exciting calculation

In [4]:
# unit cell lattice of the material
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]

# Atoms inside the unit cell: every atom own dictionary with species name and position
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]

# Structure object: pass atoms, lattice, species_path and optional kwargs as dictionary
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})

# Groundstate Object with groundstate attributes
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='fromscratch',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# define slurm run properties like timeout, partition, number of mpi nodes, ...
slurm_directives = slurm.set_slurm_directives(job_name='test1',
                                                  time=[0, 0, 5, 0],
                                                  partition='debug',
                                                  exclusive=True,
                                                  nodes=1,
                                                  ntasks_per_node=4,
                                                  cpus_per_task=4,
                                                  hint='nomultithread')

# Exciting calculation: pass name, run directory, structure, groundstate, slurm_commands
calculation1 = ExcitingSlurmCalculation('test1', 'calculation1', structure,
                                        '/mnt/beegfs2018/scratch/peschelf/code/exciting/species/', 
                                        groundstate, slurm_directives=slurm_directives)

ExcitingCalculation has three main functions to use:
1. write_inputs()
2. run()
3. parse_output()

In [5]:
calculation1.write_inputs()
print(os.listdir('calculation1'))

['F.xml', 'Li.xml', 'input.xml', 'submit_run.sh']


In [6]:
with open('calculation1/submit_run.sh', 'r') as fid:
    print(fid.read())

#!/bin/bash 

#SBATCH --job-name=test1
#SBATCH --time=0-00:05:00
#SBATCH --partition=debug
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=4
#SBATCH --cpus-per-task=4
#SBATCH --hint=nomultithread
#SBATCH --exclusive

module load intel-oneapi/2021.4.0

EXE=/mnt/beegfs2018/scratch/peschelf/code/release/exciting/bin/exciting_mpismp
OUT=terminal.out

cd ${SLURM_SUBMIT_DIR}
export OMP_NUM_THREADS=${SLURM_CPUS_PER_TASK}

# $SLURM_NTASKS = n_nodes * ntasks-per-node == total number of MPI processes
mpirun -np $SLURM_NTASKS $EXE  > $OUT


In [7]:
run_result = calculation1.run()

Put calculation into queue, JOBID=8838


In [8]:
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: []
StdErr: []
Return Code: 0
Process time: 31.21669101715088
Success: True


In [9]:
result = calculation1.parse_output()
result

{'TOTENERGY.OUT': array([-108.75147332, -107.74803581, -107.25775069, -109.01489345,
        -104.59342924, -107.21759418, -107.01836703, -107.05134603,
        -107.0507272 , -107.05202976]),
 'INFO.OUT': {'initialization': {'Lattice vectors (cartesian)': [3.804,
    0.0,
    0.0,
    0.0,
    3.804,
    0.0,
    0.0,
    0.0,
    3.804],
   'Reciprocal lattice vectors (cartesian)': [1.6517311533,
    0.0,
    0.0,
    0.0,
    1.6517311533,
    0.0,
    0.0,
    0.0,
    1.6517311533],
   'Unit cell volume': 55.045462464,
   'Brillouin zone volume': 4.5062790344,
   'Automatic determination of muffin-tin radii': '',
   'parameters': '0.2500000000      0.9500000000',
   'Species 1': {'Species symbol': 'F',
    'Species': '1 (F)',
    'parameters loaded from': 'F.xml',
    'name': 'fluorine',
    'nuclear charge': -9.0,
    'electronic charge': 9.0,
    'atomic mass': 34631.97043,
    'muffin-tin radius': 1.6514,
    '# of radial points in muffin-tin': 250,
    'Atomic positions': {'At

In [10]:
result['INFO.OUT']['initialization']['Species 1']['muffin-tin radius']

1.6514

### Exciting BSE calculation of LiF

In [11]:
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='fromscratch',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# XS part definition:
# the attributes of the xs element:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                 'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}

# define the necessary sub element trees for BSE calculation
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 100}
screening_attributes = {'screentype': 'full', 'nempty': 5}
# plan_input = ['xsgeneigvec', 'writepmatxs', 'scrgeneigvec', 'scrwritepmat', 'screen', 'scrcoulint', 'exccoulint',
#              'bse']
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]

# compose the XS object with xstype and the attributes
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

slurm_directives = slurm.set_slurm_directives(job_name='test2',
                                                  time=[0, 0, 5, 0],
                                                  partition='debug',
                                                  exclusive=True,
                                                  nodes=1,
                                                  ntasks_per_node=4,
                                                  cpus_per_task=4,
                                                  hint='nomultithread')

# BSE calculation, pass xs as last parameter
calculation2 = ExcitingSlurmCalculation('test2', 'calculation2', structure,
                                        '/mnt/beegfs2018/scratch/peschelf/code/exciting/species/', 
                                        groundstate, xs, slurm_directives)

In [12]:
calculation2.write_inputs()
with open('calculation2/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test2</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="fromscratch"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

	<xs
	   xstype="BSE"
	   broad="0.327"
	   ngridk="1 1 1"
	   nempty="5"
	   ngridq="1 1 1"
	   gqmax="2.5"
	   tappinfo="true"
	   tevout="true"
	   vkloff="0.05 0.03 0.13">

		 
		<screening screentype="full" nempty="5"> </screening>
		<BSE
		   bsetype="IP"
		   xas="true"
		   nstlxas="1 5"
		   xasatom="1"
		   xasedge="K"
		

In [13]:
run_result = calculation2.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

Put calculation into queue, JOBID=8839
StdOut: ['Info(bselauncher): Rank=   1 is idle.\n', 'Info(bselauncher): Rank=   2 is idle.\n', 'Info(bselauncher): Rank=   3 is idle.\n']
StdErr: []
Return Code: 0
Process time: 61.29093670845032
Success: True


In [14]:
result_2 = calculation2.parse_output()
print(result_2)

{'TOTENERGY.OUT': array([-108.75147332, -107.74803581, -107.25775069, -109.01489345,
       -104.59342924, -107.21759418, -107.01836703, -107.05134603,
       -107.0507272 , -107.05202976]), 'INFO.OUT': {'initialization': {'Lattice vectors (cartesian)': [3.804, 0.0, 0.0, 0.0, 3.804, 0.0, 0.0, 0.0, 3.804], 'Reciprocal lattice vectors (cartesian)': [1.6517311533, 0.0, 0.0, 0.0, 1.6517311533, 0.0, 0.0, 0.0, 1.6517311533], 'Unit cell volume': 55.045462464, 'Brillouin zone volume': 4.5062790344, 'Automatic determination of muffin-tin radii': '', 'parameters': '0.2500000000      0.9500000000', 'Species 1': {'Species symbol': 'F', 'Species': '1 (F)', 'parameters loaded from': 'F.xml', 'name': 'fluorine', 'nuclear charge': -9.0, 'electronic charge': 9.0, 'atomic mass': 34631.97043, 'muffin-tin radius': 1.6514, '# of radial points in muffin-tin': 250, 'Atomic positions': {'Atom 1': '0.00000000  0.00000000  0.00000000'}}, 'Species 2': {'Species symbol': 'Li', 'Species': '2 (Li)', 'parameters loa

### Exciting BSE calculation on top of groundstate Calculation

In [15]:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                     'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 100}
screening_attributes = {'screentype': 'full', 'nempty': 5}
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

slurm_directives = slurm.set_slurm_directives(job_name='test3',
                                                  time=[0, 0, 5, 0],
                                                  partition='debug',
                                                  exclusive=True,
                                                  nodes=1,
                                                  ntasks_per_node=4,
                                                  cpus_per_task=4,
                                                  hint='nomultithread')

calculation3 = ExcitingSlurmCalculation('test3', 'calculation3', 'calculation1', 'calculation1',
                                        'calculation1', xs, slurm_directives)

In [16]:
calculation3.write_inputs()
with open('calculation3/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test3</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0.0 0.0 0.0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="skip"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

	<xs
	   xstype="BSE"
	   broad="0.327"
	   ngridk="1 1 1"
	   nempty="5"
	   ngridq="1 1 1"
	   gqmax="2.5"
	   tappinfo="true"
	   tevout="true"
	   vkloff="0.05 0.03 0.13">

		 
		<screening screentype="full" nempty="5"> </screening>
		<BSE
		   bsetype="IP"
		   xas="true"
		   nstlxas="1 5"
		   xasatom="1"
		   xasedge="K"
		 

In [17]:
os.listdir('calculation3')

['STATE.OUT', 'EFERMI.OUT', 'F.xml', 'Li.xml', 'input.xml', 'submit_run.sh']

In [18]:
run_result = calculation3.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

Put calculation into queue, JOBID=8840
StdOut: ['Info(bselauncher): Rank=   1 is idle.\n', 'Info(bselauncher): Rank=   2 is idle.\n', 'Info(bselauncher): Rank=   3 is idle.\n']
StdErr: []
Return Code: 0
Process time: 31.255977392196655
Success: True


In [19]:
result_3 = calculation3.parse_output()
result_3

{'LOSS_BSE-IP_SCR-full_OC11.OUT': {'frequency': [1360.569193,
   1368.732608158,
   1376.896023316,
   1385.059438474,
   1393.222853632,
   1401.38626879,
   1409.549683948,
   1417.713099106,
   1425.876514264,
   1434.039929422,
   1442.20334458,
   1450.366759738,
   1458.530174896,
   1466.693590054,
   1474.857005212,
   1483.02042037,
   1491.183835528,
   1499.347250686,
   1507.510665844,
   1515.674081002,
   1523.83749616,
   1532.000911318,
   1540.164326476,
   1548.327741634,
   1556.491156792,
   1564.65457195,
   1572.817987108,
   1580.981402266,
   1589.144817424,
   1597.308232582,
   1605.47164774,
   1613.635062898,
   1621.798478056,
   1629.961893214,
   1638.125308372,
   1646.28872353,
   1654.452138688,
   1662.615553846,
   1670.778969004,
   1678.942384162,
   1687.10579932,
   1695.269214478,
   1703.432629636,
   1711.596044794,
   1719.759459952,
   1727.92287511,
   1736.086290268,
   1744.249705426,
   1752.413120584,
   1760.576535742,
   1768.7399509,

In [20]:
result_2['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength'] == result_3['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength']

True

### Using the groundstate calculation object

In [21]:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                     'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 100}
screening_attributes = {'screentype': 'full', 'nempty': 5}
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

slurm_directives = slurm.set_slurm_directives(job_name='test4',
                                                  time=[0, 0, 5, 0],
                                                  partition='debug',
                                                  exclusive=True,
                                                  nodes=1,
                                                  ntasks_per_node=4,
                                                  cpus_per_task=4,
                                                  hint='nomultithread')

calculation4 = ExcitingSlurmCalculation('test4', 'calculation4', calculation1, calculation1, calculation1,
                                        xs, slurm_directives)

In [22]:
calculation4.write_inputs()
with open('calculation4/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test4</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate
	   ngridk="4 4 4"
	   rgkmax="5.0"
	   gmaxvr="10"
	   nempty="5"
	   do="skip"
	   xctype="GGA_PBE_SOL"
	   lmaxmat="12"
	   lmaxvr="12"
	   lmaxapw="12"
	   epschg="0.1"
	   epsengy="0.1"
	   epspot="0.1">
	</groundstate>

	<xs
	   xstype="BSE"
	   broad="0.327"
	   ngridk="1 1 1"
	   nempty="5"
	   ngridq="1 1 1"
	   gqmax="2.5"
	   tappinfo="true"
	   tevout="true"
	   vkloff="0.05 0.03 0.13">

		 
		<screening screentype="full" nempty="5"> </screening>
		<BSE
		   bsetype="IP"
		   xas="true"
		   nstlxas="1 5"
		   xasatom="1"
		   xasedge="K"
		   xass

In [23]:
calculation4.run(wait_for_finish=False)

Put calculation into queue, JOBID=8841


In [24]:
try:
    run_result = calculation4.get_runresults()
except RuntimeError as RE:
    print('Got Error:', RE)

Got Error: Calculation isn't finished yet!


In [25]:
time.sleep(35)
run_result = calculation4.get_runresults()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: ['Info(bselauncher): Rank=   1 is idle.\n', 'Info(bselauncher): Rank=   2 is idle.\n', 'Info(bselauncher): Rank=   3 is idle.\n']
StdErr: []
Return Code: 0
Process time: 0
Success: True


In [26]:
result_4 = calculation4.parse_output()
result_4

{'LOSS_BSE-IP_SCR-full_OC11.OUT': {'frequency': [1360.569193,
   1368.732608158,
   1376.896023316,
   1385.059438474,
   1393.222853632,
   1401.38626879,
   1409.549683948,
   1417.713099106,
   1425.876514264,
   1434.039929422,
   1442.20334458,
   1450.366759738,
   1458.530174896,
   1466.693590054,
   1474.857005212,
   1483.02042037,
   1491.183835528,
   1499.347250686,
   1507.510665844,
   1515.674081002,
   1523.83749616,
   1532.000911318,
   1540.164326476,
   1548.327741634,
   1556.491156792,
   1564.65457195,
   1572.817987108,
   1580.981402266,
   1589.144817424,
   1597.308232582,
   1605.47164774,
   1613.635062898,
   1621.798478056,
   1629.961893214,
   1638.125308372,
   1646.28872353,
   1654.452138688,
   1662.615553846,
   1670.778969004,
   1678.942384162,
   1687.10579932,
   1695.269214478,
   1703.432629636,
   1711.596044794,
   1719.759459952,
   1727.92287511,
   1736.086290268,
   1744.249705426,
   1752.413120584,
   1760.576535742,
   1768.7399509,

In [27]:
result_4['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength'] == result_3['EPSILON_BSE-IP_SCR-full_OC11.OUT']['imag_oscillator_strength']

True

### Error:

In [28]:
# unit cell lattice of the material
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]

# Atoms inside the unit cell: every atom own dictionary with species name and position
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]

# Structure object: pass atoms, lattice, species_path and optional kwargs as dictionary
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})

# Groundstate Object with groundstate attributes
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='keks',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# define slurm run properties like timeout, partition, number of mpi nodes, ...
slurm_directives = slurm.set_slurm_directives(job_name='test5',
                                                  time=[0, 0, 5, 0],
                                                  partition='debug',
                                                  exclusive=True,
                                                  nodes=1,
                                                  ntasks_per_node=4,
                                                  cpus_per_task=4,
                                                  hint='nomultithread')

# Exciting calculation: pass name, run directory, structure, groundstate, slurm_commands
calculation5 = ExcitingSlurmCalculation('test5', 'calculation5', structure,
                                        '/mnt/beegfs2018/scratch/peschelf/code/exciting/species/', groundstate,
                                        slurm_directives=slurm_directives)

In [29]:
calculation5.write_inputs()
run_result = calculation5.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

Put calculation into queue, JOBID=8842
StdOut: [" Parser ERROR: '\n", ' keks                                                                           \n', "  ' is not valid selection for do \n", " Parser ERROR: '\n", ' keks                                                                           \n', "  ' is not valid selection for do \n", " Parser ERROR: '\n", ' keks                                                                           \n', "  ' is not valid selection for do \n", " Parser ERROR: '\n", ' keks                                                                           \n', "  ' is not valid selection for do \n"]
StdErr: []
Return Code: 0
Process time: 31.21062207221985
Success: True
